## Building a GPT

Companion notebook to the [Zero To Hero](https://karpathy.ai/zero-to-hero.html) video on GPT.

In [ ]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
!wget https://raw.githubusercontent.com/spween/gpt/master/star-wars-gpt.txt

--2023-11-16 06:43:07--  https://raw.githubusercontent.com/spween/gpt/master/star-wars-gpt.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 325816 (318K) [text/plain]
Saving to: ‘star-wars-gpt.txt’

star-wars-gpt.txt   100%[===================>] 318.18K  --.-KB/s    in 0.04s   

2023-11-16 06:43:07 (6.98 MB/s) - ‘star-wars-gpt.txt’ saved [325816/325816]



In [ ]:
# read it in to inspect it
with open('star-wars-gpt.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [ ]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  325816


In [ ]:
# let's look at the first 1000 characters
print(text[:1000])


               A long time ago, in a galaxy far, far, away...

               A vast sea of stars serves as the backdrop for the main title. 
               War drums echo through the heavens as a rollup slowly crawls 
               into infinity.

                    It is a period of civil war. Rebel spaceships, 
                    striking from a hidden base, have won their first 
                    victory against the evil Galactic Empire.

                    During the battle, Rebel spies managed to steal 
                    secret plans to the Empire's ultimate weapon, the 
                    Death Star, an armored space station with enough 
                    power to destroy an entire planet.

                    Pursued by the Empire's sinister agents, Princess 
                    Leia races home aboard her starship, custodian of 
                    the stolen plans that can save her people and 
                    restore freedom to the galaxy...

               The

In [ ]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !"'(),-./0234789:;?ABCDEFGHIJKLMNOPQRSTUVWXYZ]abcdefghijklmnopqrstuvwxyz
74


In [ ]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[55, 56, 56, 1, 67, 55, 52, 65, 52]
hii there


In [ ]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch
data = torch.tensor(encode(text), dtype = torch.long)
print(data.shape, data.type)
print(data[:1000])

torch.Size([325816]) <built-in method type of Tensor object at 0x78688e902610>
tensor([ 0,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, 21,  1,
        59, 62, 61, 54,  1, 67, 56, 60, 52,  1, 48, 54, 62,  7,  1, 56, 61,  1,
        48,  1, 54, 48, 59, 48, 71, 72,  1, 53, 48, 65,  7,  1, 53, 48, 65,  7,
         1, 48, 70, 48, 72,  9,  9,  9,  0,  0,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1, 21,  1, 69, 48, 66, 67,  1, 66, 52, 48,  1,
        62, 53,  1, 66, 67, 48, 65, 66,  1, 66, 52, 65, 69, 52, 66,  1, 48, 66,
         1, 67, 55, 52,  1, 49, 48, 50, 58, 51, 65, 62, 63,  1, 53, 62, 65,  1,
        67, 55, 52,  1, 60, 48, 56, 61,  1, 67, 56, 67, 59, 52,  9,  1,  0,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, 43, 48, 65,  1,
        51, 65, 68, 60, 66,  1, 52, 50, 55, 62,  1, 67, 55, 65, 62, 68, 54, 55,
         1, 67, 55, 52,  1, 55, 52, 48, 69, 52, 61, 66,  1, 48, 66,  1, 48,  1,
        65, 62, 59, 59, 68, 63,  1, 66, 5

In [ ]:
# Let's now split up the data into train and validation sets
# note from me: training data is for configuring the NLP model, and validation is what will be used
# to ensure the model is not being overfitted
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [ ]:
block_size = 20
train_data[:block_size+1]
# potential error? why are they all 1s? if there is an error in the model
# maybe a new data set would be better

tensor([ 0,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, 21,  1,
        59, 62, 61])

In [ ]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([0]) the target: 1
when input is tensor([0, 1]) the target: 1
when input is tensor([0, 1, 1]) the target: 1
when input is tensor([0, 1, 1, 1]) the target: 1
when input is tensor([0, 1, 1, 1, 1]) the target: 1
when input is tensor([0, 1, 1, 1, 1, 1]) the target: 1
when input is tensor([0, 1, 1, 1, 1, 1, 1]) the target: 1
when input is tensor([0, 1, 1, 1, 1, 1, 1, 1]) the target: 1
when input is tensor([0, 1, 1, 1, 1, 1, 1, 1, 1]) the target: 1
when input is tensor([0, 1, 1, 1, 1, 1, 1, 1, 1, 1]) the target: 1
when input is tensor([0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]) the target: 1
when input is tensor([0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]) the target: 1
when input is tensor([0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]) the target: 1
when input is tensor([0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]) the target: 1
when input is tensor([0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]) the target: 1
when input is tensor([0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]) the target: 21

In [ ]:
# essentially, what I did above but for the entire dataset
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[35, 23, 31, 36, 29, 40,  0,  0],
        [62, 68,  1, 55, 48, 51, 61,  4],
        [72, 62, 68,  1, 70, 52, 65, 52],
        [ 1,  1,  1,  1,  1,  1,  1,  1]])
targets:
torch.Size([4, 8])
tensor([[23, 31, 36, 29, 40,  0,  0,  1],
        [68,  1, 55, 48, 51, 61,  4, 67],
        [62, 68,  1, 70, 52, 65, 52,  1],
        [ 1,  1,  1,  1,  1,  1,  1,  1]])
----
when input is [35] the target: 23
when input is [35, 23] the target: 31
when input is [35, 23, 31] the target: 36
when input is [35, 23, 31, 36] the target: 29
when input is [35, 23, 31, 36, 29] the target: 40
when input is [35, 23, 31, 36, 29, 40] the target: 0
when input is [35, 23, 31, 36, 29, 40, 0] the target: 0
when input is [35, 23, 31, 36, 29, 40, 0, 0] the target: 1
when input is [62] the target: 68
when input is [62, 68] the target: 1
when input is [62, 68, 1] the target: 55
when input is [62, 68, 1, 55] the target: 48
when input is [62, 68, 1, 55, 48] the target: 51
when input is [62,

In [ ]:
print(xb) # our input to the transformer

tensor([[35, 23, 31, 36, 29, 40,  0,  0],
        [62, 68,  1, 55, 48, 51, 61,  4],
        [72, 62, 68,  1, 70, 52, 65, 52],
        [ 1,  1,  1,  1,  1,  1,  1,  1]])


In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


torch.Size([32, 74])
tensor(4.6010, grad_fn=<NllLossBackward0>)

7Grngn4j
7xOB.jCC9h/IZFgpF.)qKyg!v; OJ'g.xCMS3:"QOafN4Zzli3Y,rGY7!mrii h7Y8DQ-D qjTUa.nE]"U',: wIuxj


In [ ]:
# create a PyTorch optimizer (281: optimizes to limit options (backtracking and such), but takes more time)
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [ ]:
batch_size = 32
for steps in range(100): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())


4.468474864959717


In [ ]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=700)[0].tolist()))


gzY,h/49EuevCEu7Mq8WD.o.xg.aJ;"'GYe8.,ZFS
f0fa?a/zc:CQ/4AnC-QWI-FNi!sK3RbwA?n]ONq8DcvPZAjN?z')dCieQJ4y!s:tdwT?zua?Vn8PXYQGv'p.IpC8XvIfW

N]gqO/SzI:tdF R/'F/;
o(w)4F!s9oE x0kGrUn]4lBihWC2fRT;bPEeejFNDxX-x]4S/wJ(tH?NAC!Wk(dBJ:J/3?uG
y97iTmz.nSE
gRSIuXMUx]Onc:PycD?Qf/nfGAnZy":Qwh7MhZo9qKark!)v93 w,E7u9j3]gYec;'9bPEGy hFE0"B"QO(ueKZZRmb'Amt!Ai"fey;9Ma?vqecL- k!v'D:kx.2vZGy-Bqnx"BeQJge]4ifE-zj0y]A8e]gd;W
gaIE]SIu33nY0zlnnhuPHXiTF0qmK!tCDEY:Og3 "BVOJ)l-zC kC-zwIuIV!R/q9hr?PIYKpJcBHaqwPEQ4S,T v!"?f)yN?,bJ
y9ZEOQWuEQ'iX2o3Z]j)VBWJ W(uGYUHEaZfWsJHWn4i.R
7paWFgPEjAxXq"Tu w:'Dp)Z)vv2huEmPe,4t?
PI?v:D?hphZfiw
/PePE!LCkegJdejq]ak kaDM!v)oVB]?uaWEQ8;"R9Rkq:y"SgepDejTU3(dU(tDqzlvY.K(]Ye"pyaK8"T2wh/PHNI]Dc;


## The mathematical trick in self-attention

In [ ]:
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [ ]:
# consider the following toy example:

torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [ ]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)


In [ ]:
# version 2: using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
torch.allclose(xbow, xbow2)

False

In [ ]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)


False

In [ ]:
# version 4: self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
#out = wei @ x

out.shape

torch.Size([4, 8, 16])

In [ ]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

Notes:
- Attention is a **communication mechanism**. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
- There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
- Each example across batch dimension is of course processed completely independently and never "talk" to each other
- In an "encoder" attention block just delete the single line that does masking with `tril`, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
- "self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
- "Scaled" attention additional divides `wei` by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below

In [ ]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5

In [ ]:
k.var()

tensor(1.0449)

In [ ]:
q.var()

tensor(1.0700)

In [ ]:
wei.var()

tensor(1.0918)

In [ ]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [ ]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1) # gets too peaky, converges to one-hot

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])

In [ ]:
class LayerNorm1d: # (used to be BatchNorm1d)

  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)

  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(1, keepdim=True) # batch mean
    xvar = x.var(1, keepdim=True) # batch variance
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out

  def parameters(self):
    return [self.gamma, self.beta]

torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32, 100) # batch size 32 of 100-dimensional vectors
x = module(x)
x.shape

torch.Size([32, 100])

In [ ]:
x[:,0].mean(), x[:,0].std() # mean,std of one feature across all batch inputs

(tensor(0.1469), tensor(0.8803))

In [ ]:
x[0,:].mean(), x[0,:].std() # mean,std of a single input from the batch, of its features

(tensor(-9.5367e-09), tensor(1.0000))

In [ ]:
# French to English translation example:

# <--------- ENCODE ------------------><--------------- DECODE ----------------->
# les réseaux de neurones sont géniaux! <START> neural networks are awesome!<END>



### Full finished code, for reference

You may want to refer directly to the git repo instead though.

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('star-wars-gpt.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


0.21089 M parameters
step 0: train loss 4.2978, val loss 4.2945
step 100: train loss 1.8371, val loss 2.0340
step 200: train loss 1.6098, val loss 1.7827
step 300: train loss 1.5517, val loss 1.6565
step 400: train loss 1.5166, val loss 1.5850
step 500: train loss 1.4768, val loss 1.5106
step 600: train loss 1.4264, val loss 1.4857
step 700: train loss 1.3617, val loss 1.4022
step 800: train loss 1.3371, val loss 1.3639
step 900: train loss 1.3328, val loss 1.3294
step 1000: train loss 1.2818, val loss 1.2768
step 1100: train loss 1.2732, val loss 1.2530
step 1200: train loss 1.2674, val loss 1.2315
step 1300: train loss 1.2540, val loss 1.2217
step 1400: train loss 1.2267, val loss 1.1717
step 1500: train loss 1.2151, val loss 1.1751
step 1600: train loss 1.1968, val loss 1.1598
step 1700: train loss 1.1899, val loss 1.1312
step 1800: train loss 1.1481, val loss 1.1062
step 1900: train loss 1.1493, val loss 1.0984
step 2000: train loss 1.1480, val loss 1.1148
step 2100: train loss 1.1